In [1]:
!pip install pytorchvideo
!pip install torchsummary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 KB 634.2 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 KB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 21.8 MB/s eta 0:00:00
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.5-py3-none-any.whl size=188714 sha256=c62303454bedd5100e11e2c7495edeb1bd2f6a96da02a814e88b80a9eeaeb199
  Stored in directory: /root/.cache/pip/wheels/e8/51/05/053b29bac2400cbbae2fb7cfc41afd280d627bca7c9363ca80
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220512-py3-none-any.whl size=61288 sha256=295f28d804863a8b10fc20629fc30d259ea4f005957437edda5cbf890d89b0b6
  Stored in directory: /root/.cache/pip/wheels/68/20/f9/a11a0dd63f4c13678b2a5ec488e48078756505c7777b75b29e
Successfully built pytorchvideo fvcore


In [2]:
import torch
import numpy as np
from torch.utils.data import (
    Dataset,
    DataLoader,
) 
import pickle


import os

import math
import torch.nn as nn
import torch.nn.functional as F

from torchvision.transforms import Compose, Lambda, Grayscale,Normalize, CenterCrop,Resize
#from torchvision.transforms._transforms_video import CenterCropVideo
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    UniformTemporalSubsample,
)

from tqdm import tqdm

from collections import OrderedDict

import torch.optim as optim

from torch.autograd import Variable


import torchvision.models as models

import sys
from torchsummary import summary






class SignDataset(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y


    def __len__(self):

        return len(self.y)

        # length = 0
        # with open(self.file, 'rb') as f:
        #     data = pickle.load(f)

        # for x in data:
        #     length += len(data[x])
        # return length


    def __getitem__(self, index):
        return self.x[index], self.y[index]
        

In [3]:
IMAGE_HEIGHT = 720
IMAGE_WIDTH = 800
IMAGE_CHANNEL = 1
NUM_FRAMES = 25
NUM_CLASSES = 60







def transform_data(x):
    
    transform =  ApplyTransformToKey(
        key="video",
        transform=Compose(
            [
                Lambda(lambda x: x/255.0),
                Normalize(([0.2933, 0.4799, 0.6723]), ([0.1576, 0.1674, 0.2477])),
                Grayscale(num_output_channels=1),
                CenterCrop([720,900]),
                Resize([512,512]),
                Lambda(lambda x: x.permute(1,2,3,0)),#(channel, frames(depth), height, width)
#                 Lambda(lambda x: print(x.shape))

            ]

        ),
    )
    
    return transform(x)

def lp_video(video,start_time, end_time):
    global means,stds
    global nst
    
    video_data = video.get_clip(start_sec=float(start_time)/1000.0, end_sec=float(end_time)/1000.0)
            #print(video_data["video"].shape)
 
    if video_data["video"] is None:
        return None        # or pass
    else:
    
        #video_data["video"] = Grayscale(num_output_channels=1)((video_data["video"]).permute(1,0,2,3))
    #             video_data["video"] = video_data["video"]/255
                #print(video_data["video"].shape)
        video_data["video"] = video_data["video"].permute(1,0,2,3)
        
#         std, mean = torch.std_mean(video_data["video"],dim=[0,2,3])
#         std = std/255.0
#         #print(std)
#         mean = mean/255.0
#         #print(mean)
#         means += mean
#         stds +=std
#         nst += 1
        video_data = transform_data( video_data)

        return video_data["video"]

def load_dataloaders(batch_size,start,end):

    
    train_inputs =[] #x
    train_classes = [] #y
    global neg

    with open('../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_GT.pkl', 'rb') as f:
        data = pickle.load(f)


# keys are files so iterate only limited files due to memory limitations.
    for key in list(data.keys())[start:end]:
        time_start = []
        time_end = []
        filename = key
        print("file",filename)
        video = EncodedVideo.from_path("../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_VIDEOS_ELAN/"+filename+".mp4")
    # file functions
        
        for x in data[key]:
            img_cls = x[0]
            time_start.append(x[1])
            time_end.append(x[2])
            
            
            # start_time = x[1]
            # end_time = x[2]
            vid = lp_video(video,x[1], x[2])

            for m in torch.unbind(vid, dim=3):
                train_classes.append(img_cls)
                train_inputs.append(m)


            #some negative classes too
        for i in range(len(time_start)-2):
            if (time_start[i+1]- time_end[i])>1000.0:
                start_t = time_end[i]+800.0
                end_t = time_end[i]+880.0
                vid = lp_video(video,start_t, end_t)
                if vid is None:
                    break
                else:
                    for m in torch.unbind(vid, dim=3):
                        train_classes.append(60)
                        train_inputs.append(m)
            
        
            
    signds = SignDataset(train_inputs, train_classes)
    trainlen = int(len(signds)*0.8)
    torch.manual_seed(0)
    
#     train_set, val_test_set = torch.utils.data.random_split(signds, [trainlen, len(signds)-trainlen])
    trainloader = DataLoader(signds, batch_size=batch_size, shuffle=True, num_workers=2)

#     valloader = DataLoader(val_test_set, batch_size=batch_size, shuffle=True, num_workers=2)
  
    return trainloader
        
    

In [4]:
def load_testloader(batch_size,start,end):

    time_start = []
    time_end = []
    train_inputs =[] #x
    train_classes = [] #y
    

    with open('../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_GT.pkl', 'rb') as f:
        data = pickle.load(f)


# keys are files so iterate only limited files due to memory limitations.
    for key in list(data.keys())[start:end]:
        filename = key
        print("file",filename)
        video = EncodedVideo.from_path("../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_VIDEOS_ELAN/"+filename+".mp4")
    # file functions

        for x in data[key]:
            img_cls = x[0]
            time_start.append(x[1])
            time_end.append(x[2])
            
            
            # start_time = x[1]
            # end_time = x[2]
            vid = lp_video(video,x[1], x[2])

            for m in torch.unbind(vid, dim=3):
                train_classes.append(img_cls)
                train_inputs.append(m)


            #some negative classes too
        for i in range(0,len(time_start)//2):
            if (time_start[i+1]- time_end[i])>1000.0:
                start_t = time_end[i]+700.0
                end_t = time_end[i]+780.0
                vid = lp_video(video,start_t, end_t)
                if vid is None:
                    break
                else:
                    for m in torch.unbind(vid, dim=3):
                        train_classes.append(60)
                        train_inputs.append(m)

            
        
            

    signds = SignDataset(train_inputs, train_classes)
    
    

    testloader = DataLoader(signds, batch_size=batch_size, shuffle=True, num_workers=2)

    return testloader

In [5]:
def load_vid(batch_size,start,end):


    train_inputs =[] #x

    video = EncodedVideo.from_path("../input/signdataset/sign/MSSL_Val_Set/VALIDATION/MSSL_VAL_SET_VIDEOS/p01_n131.mp4")
    # file functions

    vid = lp_video(video,start, end)

    for m in torch.unbind(vid, dim=3):
        train_inputs.append(m)

    

    testloader = DataLoader(train_inputs, batch_size=batch_size, shuffle=True, num_workers=2)

    return testloader

In [6]:
"""
Creates a EfficientNetV2 Model as defined in:
Mingxing Tan, Quoc V. Le. (2021). 
EfficientNetV2: Smaller Models and Faster Training
arXiv preprint arXiv:2104.00298.
import from https://github.com/d-li14/mobilenetv2.pytorch
"""



__all__ = ['effnetv2_s', 'effnetv2_m', 'effnetv2_l', 'effnetv2_xl']


def _make_divisible(v, divisor, min_value=None):
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    :param v:
    :param divisor:
    :param min_value:
    :return:
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


# SiLU (Swish) activation function
if hasattr(nn, 'SiLU'):
    SiLU = nn.SiLU
else:
    # For compatibility with old PyTorch versions
    class SiLU(nn.Module):
        def forward(self, x):
            return x * torch.sigmoid(x)

 
class SELayer(nn.Module):
    def __init__(self, inp, oup, reduction=4):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
                nn.Linear(oup, _make_divisible(inp // reduction, 8)),
                SiLU(),
                nn.Linear(_make_divisible(inp // reduction, 8), oup),
                nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y


def conv_3x3_bn(inp, oup, stride):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
        SiLU()
    )


def conv_1x1_bn(inp, oup):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm2d(oup),
        SiLU()
    )


class MBConv(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio, use_se):
        super(MBConv, self).__init__()
        assert stride in [1, 2]

        hidden_dim = round(inp * expand_ratio)
        self.identity = stride == 1 and inp == oup
        if use_se:
            self.conv = nn.Sequential(
                # pw
                nn.Conv2d(inp, hidden_dim, 1, 1, 0, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                SELayer(inp, hidden_dim),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )
        else:
            self.conv = nn.Sequential(
                # fused
                nn.Conv2d(inp, hidden_dim, 3, stride, 1, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )


    def forward(self, x):
        if self.identity:
            return x + self.conv(x)
        else:
            return self.conv(x)


class EffNetV2(nn.Module):
    def __init__(self, cfgs, num_classes=61, width_mult=1.):
        super(EffNetV2, self).__init__()
        self.cfgs = cfgs

        # building first layer
        input_channel = _make_divisible(24 * width_mult, 8)
        layers = [conv_3x3_bn(1, input_channel, 2)]
        # building inverted residual blocks
        block = MBConv
        for t, c, n, s, use_se in self.cfgs:
            output_channel = _make_divisible(c * width_mult, 8)
            for i in range(n):
                layers.append(block(input_channel, output_channel, s if i == 0 else 1, t, use_se))
                input_channel = output_channel
        self.features = nn.Sequential(*layers)
        # building last several layers
        output_channel = _make_divisible(1792 * width_mult, 8) if width_mult > 1.0 else 1792
        self.conv = conv_1x1_bn(input_channel, output_channel)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(output_channel, num_classes)

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = self.conv(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.001)
                m.bias.data.zero_()


def effnetv2_s(**kwargs):
    """
    Constructs a EfficientNetV2-S model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  24,  2, 1, 0],
        [4,  48,  4, 2, 0],
        [4,  64,  4, 2, 0],
        [4, 128,  6, 2, 1],
        [6, 160,  9, 1, 1],
        [6, 256, 15, 2, 1],
    ]
    return EffNetV2(cfgs, **kwargs)


def effnetv2_m(**kwargs):
    """
    Constructs a EfficientNetV2-M model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  24,  3, 1, 0],
        [4,  48,  5, 2, 0],
        [4,  80,  5, 2, 0],
        [4, 160,  7, 2, 1],
        [6, 176, 14, 1, 1],
        [6, 304, 18, 2, 1],
        [6, 512,  5, 1, 1],
    ]
    return EffNetV2(cfgs, **kwargs)


def effnetv2_l(**kwargs):
    """
    Constructs a EfficientNetV2-L model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  32,  4, 1, 0],
        [4,  64,  7, 2, 0],
        [4,  96,  7, 2, 0],
        [4, 192, 10, 2, 1],
        [6, 224, 19, 1, 1],
        [6, 384, 25, 2, 1],
        [6, 640,  7, 1, 1],
    ]
    return EffNetV2(cfgs, **kwargs)


def effnetv2_xl(**kwargs):
    """
    Constructs a EfficientNetV2-XL model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  32,  4, 1, 0],
        [4,  64,  8, 2, 0],
        [4,  96,  8, 2, 0],
        [4, 192, 16, 2, 1],
        [6, 256, 24, 1, 1],
        [6, 512, 32, 2, 1],
        [6, 640,  8, 1, 1],
    ]
    return EffNetV2(cfgs, **kwargs)

In [7]:

def evluate(model, device, loader, mode="Validate"):
    
    
    model.eval()

    correct = 0
    num_samples = 0
    
        
    for batch_idx, data in enumerate(loader):
        with torch.no_grad():    
            data = data.to(device)
            

            output = model(data.half())  
            
            _,pred = output.max(1)# get the index of the max log-probability
            print(pred)



In [8]:

def test(model, device, loader, mode="Validate"):
    
    
    model.eval()

    correct = 0
    num_samples = 0
    
        
    for batch_idx, (data, target) in enumerate(loader):
        with torch.no_grad():    
            data = data.to(device)
            target = (target).to(device)

            output = model(data.half())  
            
            _,pred = output.max(1)# get the index of the max log-probability
            num_samples += target.size(0)
            correct += (pred==target).sum().item()
            #print("correct", correct)

    acc = 100.0 * correct / num_samples
    
    print('{} Accuracy: {}/{} ({:.0f}%)'.format(
                mode,correct, num_samples,
                100. * correct / num_samples, acc))

In [9]:

    


def train(model, device, train_loader,scheduler, optimizer,criterion,epochs,lr):
    print("Train start")
    breakout = False
    model.half()
    model.cuda()

    
    model.train()

    optimizer.param_groups[0]['lr'] = lr
    
    for epoch in range(epochs):
        correct = 0
        num_samples = 0
        train_loss = 0
        
        for batch_idx, (data, target) in enumerate(train_loader):
            
            data = data.to(device)
            target = target.to(device)

            optimizer.zero_grad()
            output = model(data.half())
            
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            
            train_loss += loss.item()
            
            
            _,pred = output.max(1)# get the index of the max log-probability
            num_samples += target.size(0)
            correct += (pred==target).sum().item()
            
        if optimizer.param_groups[0]['lr'] != 1e-3:    
            scheduler.step()
        #print("lr:",optimizer.param_groups[0]['lr'])
        
        
        train_loss /= num_samples
        
        if (100 * correct / num_samples) >= 95:
            
            print('Epoch: {} , Training Accuracy: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch+1, correct, num_samples,100. * correct / num_samples, train_loss))
            
           # test(model,device,validloader, mode = "Validating")
#             test(model, device, testloader, mode = "Test before Training on validation set")
           # model, optimizer = trainonval(model, device, validloader, optimizer, criterion, epochs)
#             test(model, device, testloader, mode = "Test after training on validation set")
            
            
            
            torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()}, "./model_optimizer34.pt")
            
            breakout = True
            print("breakout")
            return model, optimizer
            
            
            
       
        

        print('Epoch: {} , Training Accuracy: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch+1, correct, num_samples,
                100. * correct / num_samples, train_loss))
#         if (((epoch+1)%5) == 0) :
#             test(model,device,validloader)
#             model.train()
            
    if breakout:
            print("breakout")
            return
    
    
    
    #model, optimizer = trainonval(model, device, validloader, optimizer, criterion, epochs)
    #test(model, device, validloader, mode= "test set ")
    
    
        
    torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()}, "./model_optimizer34.pt")
    return model, optimizer
    
    
        


In [10]:
n_classes = 61


model = nn.Sequential(OrderedDict([
    ('frontend',effnetv2_m())
]))





# specify loss function (categorical cross-entropy)

# specify optimizer and learning rate
optimizer = optim.SGD(
    [
        
        {"params": model.frontend.parameters(), "lr": 1e-2},
  ],
  momentum = 0.9
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


samples_per_cls = [166.0, 370.0, 63.0, 232.0, 905.0, 102.0, 537.0, 993.0, 133.0, 154.0, 286.0, 435.0, 452.0, 779.0, 354.0, 136.0, 232.0, 2154.0, 374.0, 787.0, 337.0, 268.0, 954.0, 167.0, 543.0, 324.0, 669.0, 333.0, 211.0, 433.0, 140.0, 665.0, 107.0, 305.0, 162.0, 61.0, 1229.0, 958.0, 897.0, 543.0, 625.0, 334.0, 776.0, 111.0, 412.0, 583.0, 255.0, 103.0, 108.0, 201.0, 384.0, 632.0, 13.0, 322.0, 664.0, 435.0, 194.0, 322.0, 113.0, 462.0,2330.0]
no_of_classes=61
beta=0.9999
effective_num = 1.0 - np.power(beta, samples_per_cls)
weights = (1.0 - beta) / np.array(effective_num)
weights = weights / np.sum(weights) * no_of_classes
    

weights = torch.FloatTensor(weights).cuda()

criterion = nn.CrossEntropyLoss(weight = weights.half())

# state = torch.load("../input/sign-classification/model_optimizer34.pt")
# model.load_state_dict(state['model_state_dict'])
# model.half()
# model.cuda()
# optimizer.load_state_dict(state['optimizer_state_dict'])

In [11]:
#print('Number of model parameters: {}'.format(sum([p.data.nelement() for p in model.parameters()])))

In [12]:
#summary(model, (1, 512, 640), device='cpu')#1334+986 =2330

In [13]:
trainloader = load_dataloaders(batch_size=16,start=0, end=6)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,scheduler,optimizer,criterion,30,1e-3)

file p01_n002
file p06_n019
file p06_n007
file p01_n053
file p04_n074
file p05_n077
Train start
Epoch: 1 , Training Accuracy: 78/1710 (5%) Training Loss: 0.239065
Epoch: 2 , Training Accuracy: 111/1710 (6%) Training Loss: 0.213755
Epoch: 3 , Training Accuracy: 156/1710 (9%) Training Loss: 0.197290
Epoch: 4 , Training Accuracy: 189/1710 (11%) Training Loss: 0.184309
Epoch: 5 , Training Accuracy: 236/1710 (14%) Training Loss: 0.172478
Epoch: 6 , Training Accuracy: 348/1710 (20%) Training Loss: 0.152598
Epoch: 7 , Training Accuracy: 466/1710 (27%) Training Loss: 0.134245
Epoch: 8 , Training Accuracy: 591/1710 (35%) Training Loss: 0.116057
Epoch: 9 , Training Accuracy: 730/1710 (43%) Training Loss: 0.098180
Epoch: 10 , Training Accuracy: 839/1710 (49%) Training Loss: 0.085124
Epoch: 11 , Training Accuracy: 979/1710 (57%) Training Loss: 0.069424
Epoch: 12 , Training Accuracy: 1103/1710 (65%) Training Loss: 0.057940
Epoch: 13 , Training Accuracy: 1201/1710 (70%) Training Loss: 0.049599
Epoch

In [14]:
trainloader = load_dataloaders(batch_size=16,start=6, end=12)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,scheduler,optimizer,criterion,30,1e-3)

file p05_n027
file p05_n022
file p01_n110
file p06_n036
file p01_n069
file p06_n109
Train start
Epoch: 1 , Training Accuracy: 391/1778 (22%) Training Loss: 0.240775
Epoch: 2 , Training Accuracy: 864/1778 (49%) Training Loss: 0.121036
Epoch: 3 , Training Accuracy: 1116/1778 (63%) Training Loss: 0.084216
Epoch: 4 , Training Accuracy: 1302/1778 (73%) Training Loss: 0.058492
Epoch: 5 , Training Accuracy: 1410/1778 (79%) Training Loss: 0.044962
Epoch: 6 , Training Accuracy: 1480/1778 (83%) Training Loss: 0.034984
Epoch: 7 , Training Accuracy: 1509/1778 (85%) Training Loss: 0.028838
Epoch: 8 , Training Accuracy: 1575/1778 (89%) Training Loss: 0.020265
Epoch: 9 , Training Accuracy: 1620/1778 (91%) Training Loss: 0.018197
Epoch: 10 , Training Accuracy: 1669/1778 (94%) Training Loss: 0.013386
Epoch: 11 , Training Accuracy: 1710/1778 (96%) Training Loss: 0.010872
breakout


In [15]:
trainloader = load_dataloaders(batch_size=16,start=12, end=18)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,scheduler,optimizer,criterion,30,1e-3)

file p04_n089
file p01_n044
file p05_n037
file p04_n038
file p06_n055
file p01_n099
Train start
Epoch: 1 , Training Accuracy: 779/1726 (45%) Training Loss: 0.152698
Epoch: 2 , Training Accuracy: 1365/1726 (79%) Training Loss: 0.052371
Epoch: 3 , Training Accuracy: 1536/1726 (89%) Training Loss: 0.026756
Epoch: 4 , Training Accuracy: 1612/1726 (93%) Training Loss: 0.013165
Epoch: 5 , Training Accuracy: 1665/1726 (96%) Training Loss: 0.007257
breakout


In [16]:
trainloader = load_dataloaders(batch_size=16,start=18, end=24)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,scheduler,optimizer,criterion,30,1e-3)

file p02_n066
file p04_n035
file p01_n057
file p01_n105
file p04_n058
file p05_n043
Train start
Epoch: 1 , Training Accuracy: 714/1378 (52%) Training Loss: 0.149710
Epoch: 2 , Training Accuracy: 1182/1378 (86%) Training Loss: 0.044491
Epoch: 3 , Training Accuracy: 1261/1378 (92%) Training Loss: 0.023082
Epoch: 4 , Training Accuracy: 1280/1378 (93%) Training Loss: 0.017196
Epoch: 5 , Training Accuracy: 1322/1378 (96%) Training Loss: 0.009697
breakout


In [17]:
trainloader = load_dataloaders(batch_size=16,start=24, end=30)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,scheduler,optimizer,criterion,30,1e-3)

file p06_n020
file p02_n001
file p04_n032
file p05_n084
file p01_n102
file p06_n096
Train start
Epoch: 1 , Training Accuracy: 784/1673 (47%) Training Loss: 0.132834
Epoch: 2 , Training Accuracy: 1429/1673 (85%) Training Loss: 0.027780
Epoch: 3 , Training Accuracy: 1549/1673 (93%) Training Loss: 0.013524
Epoch: 4 , Training Accuracy: 1621/1673 (97%) Training Loss: 0.007814
breakout


In [18]:
trainloader = load_dataloaders(batch_size=16,start=30, end=36)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,scheduler,optimizer,criterion,30,1e-3)

file p02_n104
file p01_n090
file p01_n040
file p04_n073
file p01_n067
file p05_n004
Train start
Epoch: 1 , Training Accuracy: 869/1597 (54%) Training Loss: 0.111727
Epoch: 2 , Training Accuracy: 1375/1597 (86%) Training Loss: 0.025981
Epoch: 3 , Training Accuracy: 1494/1597 (94%) Training Loss: 0.013084
Epoch: 4 , Training Accuracy: 1550/1597 (97%) Training Loss: 0.009380
breakout


In [19]:
trainloader = load_dataloaders(batch_size=16,start=36, end=42)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,scheduler,optimizer,criterion,30,1e-3)

file p06_n061
file p05_n092
file p06_n015
file p04_n106
file p05_n095
file p02_n072
Train start
Epoch: 1 , Training Accuracy: 1028/1800 (57%) Training Loss: 0.101794
Epoch: 2 , Training Accuracy: 1627/1800 (90%) Training Loss: 0.016722
Epoch: 3 , Training Accuracy: 1719/1800 (96%) Training Loss: 0.007794
breakout


In [20]:
trainloader = load_dataloaders(batch_size=16,start=42, end=48)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,scheduler,optimizer,criterion,30,1e-3)

file p05_n031
file p01_n060
file p05_n048
file p05_n029
file p01_n000
file p01_n050
Train start
Epoch: 1 , Training Accuracy: 885/1525 (58%) Training Loss: 0.125260
Epoch: 2 , Training Accuracy: 1359/1525 (89%) Training Loss: 0.020447
Epoch: 3 , Training Accuracy: 1453/1525 (95%) Training Loss: 0.008610
breakout


In [21]:
trainloader = load_dataloaders(batch_size=16,start=48, end=54)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,scheduler,optimizer,criterion,30,1e-3)

file p05_n025
file p04_n011
file p01_n023
file p01_n042
file p01_n111
file p05_n083
Train start
Epoch: 1 , Training Accuracy: 856/1502 (57%) Training Loss: 0.108837
Epoch: 2 , Training Accuracy: 1340/1502 (89%) Training Loss: 0.023798
Epoch: 3 , Training Accuracy: 1425/1502 (95%) Training Loss: 0.018582
Epoch: 4 , Training Accuracy: 1439/1502 (96%) Training Loss: 0.014398
breakout


In [22]:
trainloader = load_dataloaders(batch_size=16,start=54, end=60)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,scheduler,optimizer,criterion,30,1e-3)

file p05_n094
file p04_n103
file p01_n093
file p04_n003
file p05_n078
file p01_n052
Train start
Epoch: 1 , Training Accuracy: 646/1000 (65%) Training Loss: 0.084154
Epoch: 2 , Training Accuracy: 928/1000 (93%) Training Loss: 0.011814
Epoch: 3 , Training Accuracy: 969/1000 (97%) Training Loss: 0.006261
breakout


In [23]:
trainloader = load_dataloaders(batch_size=16,start=60, end=66)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,scheduler,optimizer,criterion,30,1e-3)

file p02_n098
file p06_n013
file p05_n006
file p01_n063
file p02_n097
file p05_n065
Train start
Epoch: 1 , Training Accuracy: 985/1783 (55%) Training Loss: 0.095674
Epoch: 2 , Training Accuracy: 1589/1783 (89%) Training Loss: 0.017318
Epoch: 3 , Training Accuracy: 1728/1783 (97%) Training Loss: 0.007770
breakout


In [24]:
trainloader = load_dataloaders(batch_size=16,start=66, end=72)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,scheduler,optimizer,criterion,30,1e-3)

file p01_n108
file p06_n018
file p05_n009
file p05_n071
file p04_n091
file p05_n075
Train start
Epoch: 1 , Training Accuracy: 710/1036 (69%) Training Loss: 0.085804
Epoch: 2 , Training Accuracy: 984/1036 (95%) Training Loss: 0.011048
Epoch: 3 , Training Accuracy: 1003/1036 (97%) Training Loss: 0.006574
breakout


In [25]:
trainloader = load_dataloaders(batch_size=16,start=72, end=78)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,scheduler,optimizer,criterion,30,1e-3)

file p01_n012
file p05_n076
file p04_n005
file p04_n046
file p01_n045
file p05_n041
Train start
Epoch: 1 , Training Accuracy: 1099/1620 (68%) Training Loss: 0.077422
Epoch: 2 , Training Accuracy: 1512/1620 (93%) Training Loss: 0.010693
Epoch: 3 , Training Accuracy: 1581/1620 (98%) Training Loss: 0.005313
breakout


In [26]:
trainloader = load_dataloaders(batch_size=16,start=78, end=84)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,scheduler,optimizer,criterion,30,1e-3)

file p01_n008
file p02_n101
file p01_n085
file p05_n064
file p05_n017
file p01_n028
Train start
Epoch: 1 , Training Accuracy: 1035/1666 (62%) Training Loss: 0.094722
Epoch: 2 , Training Accuracy: 1531/1666 (92%) Training Loss: 0.013871
Epoch: 3 , Training Accuracy: 1607/1666 (96%) Training Loss: 0.008048
breakout


In [27]:
trainloader = load_dataloaders(batch_size=16,start=84, end=90)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,scheduler,optimizer,criterion,30,1e-3)

file p02_n079
file p01_n010
file p01_n082
file p01_n107
file p02_n088
file p01_n087
Train start
Epoch: 1 , Training Accuracy: 703/1433 (49%) Training Loss: 0.138630
Epoch: 2 , Training Accuracy: 1194/1433 (83%) Training Loss: 0.025169
Epoch: 3 , Training Accuracy: 1354/1433 (94%) Training Loss: 0.010094
Epoch: 4 , Training Accuracy: 1396/1433 (97%) Training Loss: 0.005477
breakout


In [28]:
trainloader = load_dataloaders(batch_size=16,start=90, end=96)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,scheduler,optimizer,criterion,30,1e-3)

file p05_n086
file p06_n024
file p05_n070
file p04_n016
file p04_n068
file p04_n026
Train start
Epoch: 1 , Training Accuracy: 956/1441 (66%) Training Loss: 0.085118
Epoch: 2 , Training Accuracy: 1341/1441 (93%) Training Loss: 0.013862
Epoch: 3 , Training Accuracy: 1401/1441 (97%) Training Loss: 0.009330
breakout


In [29]:
trainloader = load_dataloaders(batch_size=16,start=96, end=102)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,scheduler,optimizer,criterion,30,1e-3)

file p06_n039
file p06_n014
file p05_n059
file p02_n051
file p06_n056
file p04_n080
Train start
Epoch: 1 , Training Accuracy: 870/1394 (62%) Training Loss: 0.089716
Epoch: 2 , Training Accuracy: 1285/1394 (92%) Training Loss: 0.014932
Epoch: 3 , Training Accuracy: 1347/1394 (97%) Training Loss: 0.006831
breakout


In [30]:
trainloader = load_dataloaders(batch_size=16,start=102, end=108)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,scheduler,optimizer,criterion,30,1e-3)

file p05_n062
file p04_n034
file p04_n049
file p06_n033
file p02_n100
file p04_n030
Train start
Epoch: 1 , Training Accuracy: 854/1250 (68%) Training Loss: 0.080728
Epoch: 2 , Training Accuracy: 1181/1250 (94%) Training Loss: 0.010762
Epoch: 3 , Training Accuracy: 1215/1250 (97%) Training Loss: 0.007015
breakout


In [31]:
trainloader = load_dataloaders(batch_size=16,start=108, end=112)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,scheduler,optimizer,criterion,30,1e-3)

file p06_n054
file p05_n047
file p06_n021
file p01_n081
Train start
Epoch: 1 , Training Accuracy: 667/1124 (59%) Training Loss: 0.090490
Epoch: 2 , Training Accuracy: 992/1124 (88%) Training Loss: 0.013405
Epoch: 3 , Training Accuracy: 1070/1124 (95%) Training Loss: 0.007672
breakout


In [32]:
trainloader = load_testloader(batch_size=16,start=0, end=6)#73 not included 
test(model,device,trainloader)

file p01_n002
file p06_n019
file p06_n007
file p01_n053
file p04_n074
file p05_n077
Validate Accuracy: 349/1831 (19%)


In [33]:
trainloader= load_vid(batch_size=16,start=3000, end=7000)#73 not included 
evluate(model,device,trainloader)

tensor([60, 60, 50, 50, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60],
       device='cuda:0')
tensor([60, 60, 60, 60, 60,  8, 60, 20,  8, 60,  8, 60, 60, 50, 60, 60],
       device='cuda:0')
tensor([60, 60, 20, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 20],
       device='cuda:0')
tensor([50, 60, 60, 60, 60, 60, 13,  8, 60,  8,  8, 60, 60, 60, 60, 20],
       device='cuda:0')
tensor([60,  8,  8, 60, 60, 45, 60, 60, 60, 60, 12, 60, 45, 60, 60, 60],
       device='cuda:0')
tensor([60, 60, 60,  8, 60, 50, 60, 19, 60, 60, 60, 60, 60, 60, 60, 19],
       device='cuda:0')
tensor([60, 60, 60, 60], device='cuda:0')
